In [10]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'vqa.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [11]:
df['cleaning_status'].value_counts()

cleaning_status
revised    242
Name: count, dtype: int64

In [12]:
df.head()

,cleaning_status,image_path,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_stratagy,question_type,multiple_choice_answer,answers,image_id,answer_type,question_id,question
0,revised,val2014/COCO_val2014_000000304481.jpg,Is it cold outside? Answer with either 'yes' o...,Is it cold outside? Answer with either 'yes' o...,[yes],[],vqa,is it,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",304481,yes/no,304481002,Is it cold outside?
1,revised,val2014/COCO_val2014_000000219984.jpg,Is it cold outside? Answer with either 'yes' o...,Is it cold outside? Answer with either 'yes' o...,[no],[],vqa,is it,no,"[{'answer': 'no', 'answer_confidence': 'no', '...",219984,yes/no,219984008,Is it cold outside?
2,revised,val2014/COCO_val2014_000000566049.jpg,Is she throwing a frisbee? Answer with either ...,Is she throwing a frisbee? Answer with either ...,[yes],[],vqa,is,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",566049,yes/no,566049012,Is she throwing a frisbee?
3,revised,val2014/COCO_val2014_000000514598.jpg,Is she throwing a frisbee? Answer with either ...,Is she throwing a frisbee? Answer with either ...,[no],[],vqa,is,no,"[{'answer': 'no', 'answer_confidence': 'yes', ...",514598,yes/no,514598006,Is she throwing a frisbee?
4,revised,val2014/COCO_val2014_000000420330.jpg,Is there cheese in that dish? Answer with eith...,Is there cheese in that dish? Answer with eith...,[yes],[],vqa,is there,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",420330,yes/no,420330002,Is there cheese in that dish?


In [13]:
df['platinum_prompt'][1]

"Is it cold outside? Answer with either 'yes' or 'no'."

In [19]:
import pandas as pd
import re

# 1. Read the question again: ...
# def repeat_question(match):
#     # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
#     # match.group(2): 질문 텍스트
#     # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
#     # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
#     # 두 번째 줄바꿈을 그대로 붙인다.
#     return f"{match.group(1)}{match.group(2)}\n\nRead the question again: Think step by step. {match.group(2)}{match.group(3)}"

# # \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
# df['RE2'] = df['platinum_prompt'].str.replace(
#     r'(\n\s*\n)(.*?)(\n\s*\n)(?=Answer with either)',
#     repeat_question,
#     flags=re.DOTALL,
#     regex=True
# )

def transform_prompt(match):
    # group 1: 질문 부분 (첫 번째 물음표까지, 예: "Is it cold outside?")
    # group 2: 질문과 나머지 사이의 공백 (있을 경우)
    # group 3: 나머지 부분 (예: "Answer with either 'yes' or 'no'.")
    q = match.group(1)
    space = match.group(2)
    rest = match.group(3)
    return f"\"{q}{space}\n\nRead the question again: {q}\n\nThink step by step. {rest}\""

df['RE2'] = df['platinum_prompt'].str.replace(
    r'^(.*?\?)(\s*)(.*)$',
    transform_prompt,
    regex=True
)

# 2. Summarize the question first.
df['sum'] = df['platinum_prompt'].str.replace(
    r'(?=Answer with either)',
    "Summarize the question first. Think step by step. ",
    regex=True
)

# 3. Organize the question into a table first.
df['table'] = df['platinum_prompt'].str.replace(
    r'(?=Answer with either)',
    "Organize the question into a table first. Think step by step. ",
    regex=True
)

# 4. Organize the question into a graph structure first.
df['graph'] = df['platinum_prompt'].str.replace(
    r'(?=Answer with either)',
    "Organize the question into a graph structure first. Think step by step. ",
    regex=True
)

# 5. Summarize the question in bullet points first.
df['bullet_point'] = df['platinum_prompt'].str.replace(
    r'(?=Answer with either)',
    "Summarize the question in bullet points first. Think step by step. ",
    regex=True
)

# 6. Read the question again first.
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(?=Answer with either)',
    "Read the question again first. Think step by step. ",
    regex=True
)

In [20]:
df['RE2'][0]

'"Is it cold outside? \n\nRead the question again: Is it cold outside?\n\nThink step by step. Answer with either \'yes\' or \'no\'."'

In [21]:
df['platinum_prompt_no_cot'][0]

"Is it cold outside? Answer with either 'yes' or 'no'."

In [ ]:
import pandas as pd
import re

# def repeat_question(match):
#     # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
#     # match.group(2): 질문 텍스트
#     # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
#     # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
#     # 두 번째 줄바꿈을 그대로 붙인다.
#     return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# # \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
# df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
#     r'(\n\s*\n)(.*?)(\n\s*\n)(?=Then, provide the final answer in the format)',
#     repeat_question,
#     flags=re.DOTALL,
#     regex=True
# )

def transform_prompt(match):
    # group 1: 질문 부분 (첫 번째 물음표까지, 예: "Is it cold outside?")
    # group 2: 질문과 나머지 사이의 공백 (있을 경우)
    # group 3: 나머지 부분 (예: "Answer with either 'yes' or 'no'.")
    q = match.group(1)
    space = match.group(2)
    rest = match.group(3)
    return f"\"{q}{space}\n\nRead the question again: {q}\n\n{rest}\""

df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'^(.*?\?)(\s*)(.*)$',
    transform_prompt,
    regex=True
)

df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Answer with either)',
    "Summarize the question first. ",
    regex=True
)

df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Answer with either)',
    "Organize the question into a table first. ",
    regex=True
)

df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Answer with either)',
    "Organize the question into a graph structure first. ",
    regex=True
)

df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Answer with either)',
    "Summarize the question in bullet points first. ",
    regex=True
)

df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Answer with either)',
    "Read the question again first. ",
    regex=True
)

In [23]:
df['RE2_no_cot'][0]

KeyError: 'RE2_no_cot'

In [12]:
df.to_parquet('tab_fact_add_task.parquet')

In [13]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,id,table_id,table_text,table_caption,...,table,graph,bullet_point,sRE2,RE2_no_cot,sum_no_cot,table_no_cot,graph_no_cot,bullet_point_no_cot,sRE2_no_cot
0,verified,Consider the following table.\n\nTable:\n| tea...,Consider the following table.\n\nTable:\n| tea...,[0],[0],text,1249,2-16770037-3.html.csv,team#tries for#tries against#try diff#points f...,1996 - 97 european challenge cup,...,Consider the following table.\n\nTable:\n| tea...,Consider the following table.\n\nTable:\n| tea...,Consider the following table.\n\nTable:\n| tea...,Consider the following table.\n\nTable:\n| tea...,Consider the following table.\n\nTable:\n| tea...,Consider the following table.\n\nTable:\n| tea...,Consider the following table.\n\nTable:\n| tea...,Consider the following table.\n\nTable:\n| tea...,Consider the following table.\n\nTable:\n| tea...,Consider the following table.\n\nTable:\n| tea...
1,verified,Consider the following table.\n\nTable:\n| g...,Consider the following table.\n\nTable:\n| g...,[0],[0],text,182,2-14148130-1.html.csv,games#drawn#lost#points difference#points\n7#0...,1983 world ice hockey championships,...,Consider the following table.\n\nTable:\n| g...,Consider the following table.\n\nTable:\n| g...,Consider the following table.\n\nTable:\n| g...,Consider the following table.\n\nTable:\n| g...,Consider the following table.\n\nTable:\n| g...,Consider the following table.\n\nTable:\n| g...,Consider the following table.\n\nTable:\n| g...,Consider the following table.\n\nTable:\n| g...,Consider the following table.\n\nTable:\n| g...,Consider the following table.\n\nTable:\n| g...
2,verified,Consider the following table.\n\nTable:\n| dat...,Consider the following table.\n\nTable:\n| dat...,[1],[1],text,40,2-1564278-3.html.csv,date#tournament#surface#opponent in the final#...,alex bogdanovic,...,Consider the following table.\n\nTable:\n| dat...,Consider the following table.\n\nTable:\n| dat...,Consider the following table.\n\nTable:\n| dat...,Consider the following table.\n\nTable:\n| dat...,Consider the following table.\n\nTable:\n| dat...,Consider the following table.\n\nTable:\n| dat...,Consider the following table.\n\nTable:\n| dat...,Consider the following table.\n\nTable:\n| dat...,Consider the following table.\n\nTable:\n| dat...,Consider the following table.\n\nTable:\n| dat...
3,rejected,Consider the following table.\n\nTable:\n| nam...,Consider the following table.\n\nTable:\n| nam...,None,[1],text,1578,2-15463188-1.html.csv,name#position#number#school / club team#season...,none,...,Consider the following table.\n\nTable:\n| nam...,Consider the following table.\n\nTable:\n| nam...,Consider the following table.\n\nTable:\n| nam...,Consider the following table.\n\nTable:\n| nam...,Consider the following table.\n\nTable:\n| nam...,Consider the following table.\n\nTable:\n| nam...,Consider the following table.\n\nTable:\n| nam...,Consider the following table.\n\nTable:\n| nam...,Consider the following table.\n\nTable:\n| nam...,Consider the following table.\n\nTable:\n| nam...
4,verified,Consider the following table.\n\nTable:\n| out...,Consider the following table.\n\nTable:\n| out...,[1],[1],text,451,2-11307139-5.html.csv,outcome#date#tournament#surface#opponent#score...,aleksandra wozniak,...,Consider the following table.\n\nTable:\n| out...,Consider the following table.\n\nTable:\n| out...,Consider the following table.\n\nTable:\n| out...,Consider the following table.\n\nTable:\n| out...,Consider the following table.\n\nTable:\n| out...,Consider the following table.\n\nTable:\n| out...,Consider the following table.\n\nTable:\n| out...,Consider the following table.\n\nTable:\n| out...,Consider the following table.\n\nTable:\n| out...,Consider the following table.\n\nTable:\n| out...
